## 1. Execute full pipeline

In [1]:
import random
import time

from keras_tuner import RandomSearch

import numpy as np

import pandas as pd

import tensorflow as tf

from config.constants import (
    FORECAST_HORIZON, FORECASTER_MODEL,
    NB_TRIALS, OBSERVATION_WINDOW, SEED, TRAIN_PERC
)

from src.cut_point_detector import CutPointMethod, CutPointModel, get_cut_point_detector
from src.dataset import read_dataset, split_X_y, split_train_test
from src.forecaster import InternalForecaster, TimeSeriesHyperModel
from src.scaler import Scaler
from src.utils import get_error_results

tf.get_logger().setLevel('ERROR')
tf.keras.mixed_precision.set_global_policy("mixed_float16")

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
timestamp = 'validate_pipeline'
dataset_domain_argv = 'UCI'
dataset_argv = 'AIR_QUALITY'
cut_point_model_argv = 'Window'
cut_point_method_argv = 'L1'

In [3]:
execution_id = f"{timestamp}_{dataset_domain_argv}_{dataset_argv}_{cut_point_model_argv}_{cut_point_method_argv}_{SEED}"

In [4]:
print(f"Extracting cut point model enum ({cut_point_model_argv})")
cut_point_model = CutPointModel.from_str(cut_point_model_argv)

print(f"Extracting cut point model enum ({cut_point_method_argv})")
cut_point_method = CutPointMethod.from_str(cut_point_method_argv)

print(f"Reading dataset {dataset_argv} from {dataset_domain_argv}")
df, variables = read_dataset(dataset_domain_argv, dataset_argv)
print(f"Variables: {variables}")

Extracting cut point model enum (Window)
Extracting cut point model enum (L1)
Reading dataset AIR_QUALITY from UCI
Variables: ['C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH', 'AH']


In [5]:
print("Splitting data into train and test")
train, test = split_train_test(df)

print("Initializing report")
cut_point_approach = f"{cut_point_model.value.title()} {cut_point_method.value.title()}"
report = {
    'execution_id': execution_id,
    'timestamp': timestamp,
    'cut_point_model': cut_point_model.value,
    'cut_point_method': cut_point_method.value,
    'cut_point_approach': cut_point_approach,
    'seed': SEED,
    'forecaster_model': FORECASTER_MODEL,
    'observation_window': OBSERVATION_WINDOW,
    'train_perc': TRAIN_PERC,
    'nb_trials': NB_TRIALS,
    'dataset_domain': dataset_domain_argv,
    'dataset': dataset_argv,
    'variables': variables,
    'dataset_shape': df.shape,
    'train_shape': train.shape,
    'test_shape': test.shape,
}
report

Splitting data into train and test
Initializing report


{'execution_id': 'validate_pipeline_UCI_AIR_QUALITY_Window_L1_42',
 'timestamp': 'validate_pipeline',
 'cut_point_model': 'Window',
 'cut_point_method': 'L1',
 'cut_point_approach': 'Window L1',
 'seed': 42,
 'forecaster_model': 'LSTM',
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'AIR_QUALITY',
 'variables': ['C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH', 'AH'],
 'dataset_shape': (9357, 7),
 'train_shape': (7485, 7),
 'test_shape': (1872, 7)}

In [6]:
print(f"Started cut point for {cut_point_approach}")
start_time = time.time()
cut_point_detector = get_cut_point_detector(cut_point_model, cut_point_method)
cut_point, cut_point_perc = cut_point_detector.find_cut_point(train, variables)
end_time = time.time()
cut_duration = end_time - start_time
print(f"Cut point: {cut_point}, Cut point percentage: {cut_point_perc}")
print(f"Finished cut point for {cut_point_approach}, duration: {cut_duration}")

report.update({
    'cut_duration': cut_duration,
    'cut_point': str(cut_point),
    'cut_point_perc': cut_point_perc
})
report

Started cut point for Window L1
Cut point: 5385, Cut point percentage: 71.9438877755511
Finished cut point for Window L1, duration: 0.10567998886108398


{'execution_id': 'validate_pipeline_UCI_AIR_QUALITY_Window_L1_42',
 'timestamp': 'validate_pipeline',
 'cut_point_model': 'Window',
 'cut_point_method': 'L1',
 'cut_point_approach': 'Window L1',
 'seed': 42,
 'forecaster_model': 'LSTM',
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'AIR_QUALITY',
 'variables': ['C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH', 'AH'],
 'dataset_shape': (9357, 7),
 'train_shape': (7485, 7),
 'test_shape': (1872, 7),
 'cut_duration': 0.10567998886108398,
 'cut_point': '5385',
 'cut_point_perc': 71.9438877755511}

In [7]:
print("Applying subset to train based on cut point")
reduced_train = cut_point_detector.apply_cut_point(train, cut_point)

print("Training and applying scaler")
scaler = Scaler(variables)
scaled_reduced_train = scaler.fit_scale(reduced_train)
scaled_test = scaler.scale(test)

Applying subset to train based on cut point
Training and applying scaler


In [8]:
print("Splitting into X and y")
X_reduced_scaled_train, y_reduced_scaled_train = split_X_y(scaled_reduced_train)
X_scaled_test, y_scaled_test = split_X_y(scaled_test)

Splitting into X and y


In [9]:
y_reduced_scaled_train[0]

array([[0.32598425, 0.21964529, 0.32      , 0.99230769, 0.49531459,
        0.87453389],
       [0.4488189 , 0.29604366, 0.38181818, 0.95      , 0.55957162,
        0.89707173],
       [0.54488189, 0.3840382 , 0.38909091, 0.91923077, 0.60776439,
        0.91270015],
       [0.42992126, 0.32264666, 0.34181818, 0.90384615, 0.59839357,
        0.87809827],
       [0.38897638, 0.29058663, 0.33090909, 0.86153846, 0.64524766,
        0.87053082],
       [0.37795276, 0.30627558, 0.28363636, 0.81538462, 0.71084337,
        0.87475324],
       [0.34645669, 0.29195089, 0.28363636, 0.78461538, 0.73895582,
        0.85775389]])

In [10]:
y_reduced_scaled_train[1]

array([[0.4488189 , 0.29604366, 0.38181818, 0.95      , 0.55957162,
        0.89707173],
       [0.54488189, 0.3840382 , 0.38909091, 0.91923077, 0.60776439,
        0.91270015],
       [0.42992126, 0.32264666, 0.34181818, 0.90384615, 0.59839357,
        0.87809827],
       [0.38897638, 0.29058663, 0.33090909, 0.86153846, 0.64524766,
        0.87053082],
       [0.37795276, 0.30627558, 0.28363636, 0.81538462, 0.71084337,
        0.87475324],
       [0.34645669, 0.29195089, 0.28363636, 0.78461538, 0.73895582,
        0.85775389],
       [0.20472441, 0.14324693, 0.22181818, 0.77307692, 0.72958501,
        0.83220004]])

In [11]:
y_reduced_scaled_train[2]

array([[0.54488189, 0.3840382 , 0.38909091, 0.91923077, 0.60776439,
        0.91270015],
       [0.42992126, 0.32264666, 0.34181818, 0.90384615, 0.59839357,
        0.87809827],
       [0.38897638, 0.29058663, 0.33090909, 0.86153846, 0.64524766,
        0.87053082],
       [0.37795276, 0.30627558, 0.28363636, 0.81538462, 0.71084337,
        0.87475324],
       [0.34645669, 0.29195089, 0.28363636, 0.78461538, 0.73895582,
        0.85775389],
       [0.20472441, 0.14324693, 0.22181818, 0.77307692, 0.72958501,
        0.83220004],
       [0.12125984, 0.07708049, 0.18181818, 0.76923077, 0.73092369,
        0.82572933]])

In [12]:
print(f"Started running HPO and NAS for {cut_point_approach}")
forecaster_hypermodel = TimeSeriesHyperModel(
    model_type=FORECASTER_MODEL,
    n_variables=len(variables)
)
forecaster_tuner = RandomSearch(
    forecaster_hypermodel,
    objective='val_loss',
    max_trials=NB_TRIALS,
    executions_per_trial=1,
    directory=f"outputs/tuner/{execution_id}",
    project_name=f"{cut_point_model.value}_{cut_point_method.value}",
    seed=SEED,
    overwrite=True,
    distribution_strategy=tf.distribute.MirroredStrategy()
)

Started running HPO and NAS for Window L1


2025-02-25 12:14:01.844288: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-02-25 12:14:01.844323: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-25 12:14:01.844333: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-25 12:14:01.844356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-25 12:14:01.844371: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
start_time = time.time()
forecaster_tuner.search(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    validation_split=(1 - TRAIN_PERC),
    shuffle=False,
)
end_time = time.time()
tuner_duration = end_time - start_time

Trial 15 Complete [00h 00m 13s]
val_loss: 0.013428425299935043

Best val_loss So Far: 0.00867647328414023
Total elapsed time: 00h 11m 10s


In [14]:
best_trial = forecaster_tuner.oracle.get_best_trials(num_trials=1)[0]
best_forecaster_model = forecaster_tuner.get_best_models(num_models=1)[0]
print(f"Finished running HPO and NAS for {cut_point_approach}, duration: {tuner_duration}")

print(f"Trial ID: {best_trial.trial_id}")
print(f"Hyperparameters: {best_trial.hyperparameters.values}")
print(f"Score: {best_trial.score}")
print("-" * 40)

Finished running HPO and NAS for Window L1, duration: 670.1102960109711
Trial ID: 06
Hyperparameters: {'num_layers': 1, 'units_0': 128, 'learning_rate': 0.01, 'units_1': 64, 'units_2': 32, 'units_3': 32, 'batch_size': 16, 'epochs': 225}
Score: 0.00867647328414023
----------------------------------------


/Users/zamith/Doutorado/TSForecastingSubsets/env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
print("Retrieving best model")
best_forecaster_model.summary()
best_forecaster_model = InternalForecaster(
    best_forecaster_model,
    len(variables),
    best_trial.hyperparameters.values['batch_size'],
    best_trial.hyperparameters.values['epochs'],
)

Retrieving best model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 42)             │         5,418 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 6)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,538 (291.16 KB)

 Trainable params: 74,538 (291.16 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
print("Retraining best model")
start_time = time.time()
best_forecaster_model.fit(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    shuffle=False
)
end_time = time.time()
retrain_duration = end_time - start_time

Retraining best model
Epoch 1/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0125
Epoch 2/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0127
Epoch 3/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0124
Epoch 4/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0115
Epoch 5/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0113
Epoch 6/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0107
Epoch 7/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0106
Epoch 8/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0101
Epoch 9/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0100
Epoch 10/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0099
Epoch 11/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0100
Epoch 12/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0097
Epoch 13/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0094
Epoch 14/225
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0091
Epoch 15

In [17]:
print("Running forecasting")
y_scaled_pred = best_forecaster_model.forecast(X_scaled_test)
y_scaled_test_flat = y_scaled_test.reshape(-1, len(variables))
y_scaled_pred_flat = y_scaled_pred.reshape(-1, len(variables))

Running forecasting
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


2025-02-25 12:26:13.714428: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2025-02-25 12:26:13.714445: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


In [18]:
y_scaled_test_flat

array([[ 0.01259843,  0.05525239,  0.24      ,  0.05384615,  0.59705489,
         0.1314433 ],
       [ 0.02204724,  0.07366985,  0.28      ,  0.07692308,  0.562249  ,
         0.13029173],
       [ 0.04724409,  0.08049113,  0.29818182,  0.09230769,  0.55421687,
         0.13330774],
       ...,
       [ 0.19212598,  0.19099591,  0.58909091,  0.98846154,  0.05756359,
         0.24226804],
       [ 0.14645669,  0.15143247,  0.52      ,  1.04230769, -0.00669344,
         0.17279009],
       [ 0.18425197,  0.17189632,  0.56363636,  1.05      , -0.01204819,
         0.16670322]])

In [19]:
y_scaled_pred_flat

array([[-0.03756991,  0.09853232,  0.2508643 ,  0.01759565,  0.5965497 ,
         0.0983094 ],
       [ 0.0369684 ,  0.15940697,  0.3328989 ,  0.0395399 ,  0.5829699 ,
         0.09887704],
       [ 0.12583026,  0.24291866,  0.4204381 ,  0.08346581,  0.55507225,
         0.1022462 ],
       ...,
       [ 0.17316769,  0.29553118,  0.5638131 ,  0.49880123,  0.43758625,
         0.28870514],
       [ 0.18795998,  0.28757802,  0.55704725,  0.50333244,  0.44066167,
         0.28953692],
       [ 0.19758046,  0.29473448,  0.557574  ,  0.5027285 ,  0.44965523,
         0.29291803]], dtype=float32)

In [20]:
print("Calculating error")
y_test = scaler.descale(pd.DataFrame(y_scaled_test_flat, columns=variables))
y_pred = scaler.descale(pd.DataFrame(y_scaled_pred_flat, columns=variables))

Calculating error


In [21]:
y_test

,C6H6(GT),NOx(GT),NO2(GT),T,RH,AH
0,1.0,94.0,79.0,2.6,58.6,0.4385
1,1.6,121.0,90.0,3.2,56.0,0.4364
2,3.2,131.0,95.0,3.6,55.4,0.4419
3,15.9,552.0,169.0,1.8,63.3,0.4464
4,18.0,614.0,199.0,2.4,60.2,0.4426
...,...,...,...,...,...,...
12959,13.5,472.0,190.0,21.9,29.3,0.7568
12960,11.4,353.0,179.0,24.3,23.7,0.7119
12961,12.4,293.0,175.0,26.9,18.3,0.6406
12962,9.5,235.0,156.0,28.3,13.5,0.5139


In [22]:
y_pred

,C6H6(GT),NOx(GT),NO2(GT),T,RH,AH
0,-2.185689,157.448380,81.987686,1.657487,58.562260,0.378077
1,2.547493,246.690628,104.547203,2.228037,57.547852,0.379112
2,8.190222,369.118744,128.620483,3.370111,55.463894,0.385256
3,13.002934,465.940002,149.018234,4.650496,52.119324,0.404378
4,15.624730,503.437195,159.663971,6.210490,47.828659,0.426667
...,...,...,...,...,...,...
12959,13.256338,577.499756,177.529587,13.039806,50.150066,0.736171
12960,10.961380,497.625671,172.765152,13.721872,48.017483,0.725088
12961,11.196149,446.248749,168.048599,14.168832,46.687691,0.725283
12962,12.135460,434.589386,166.188004,14.286643,46.917427,0.726800


In [23]:
len(X_scaled_test)

1852

In [24]:
len(X_scaled_test) * FORECAST_HORIZON

12964

In [25]:
error_results = get_error_results(y_test, y_pred, variables)
print(f"Obtained error results: {error_results}")

Obtained error results: {'Avg_MAPE': 735572565943.2211, 'Avg_MAE': 32.998345925431174, 'Avg_MSE': 6819.226338381778, 'Avg_RMSE': 44.04242264408996, 'Avg_R2': 0.40560638760242645, 'Avg_WAPE': 0.3864814545225475, 'C6H6(GT)_MAPE': 1.2469010335105057, 'C6H6(GT)_MAE': 4.368933224942249, 'C6H6(GT)_MSE': 34.76875543181349, 'C6H6(GT)_RMSE': 5.896503661646746, 'C6H6(GT)_R2': 0.16898178581068024, 'C6H6(GT)_WAPE': 0.5460463650221601, 'NOx(GT)_MAPE': 0.7467531834453196, 'NOx(GT)_MAE': 147.12395541165165, 'NOx(GT)_MSE': 38583.16303004005, 'NOx(GT)_RMSE': 196.42597340993387, 'NOx(GT)_R2': 0.11910252882921102, 'NOx(GT)_WAPE': 0.48845606989086254, 'NO2(GT)_MAPE': 0.29137297811552104, 'NO2(GT)_MAE': 34.78528228068345, 'NO2(GT)_MSE': 2138.9093553972684, 'NO2(GT)_RMSE': 46.24834435303894, 'NO2(GT)_R2': 0.26874799042776576, 'NO2(GT)_WAPE': 0.24375606549768486, 'T_MAPE': 4413435395656.625, 'T_MAE': 2.4420369354117906, 'T_MSE': 11.90236901787607, 'T_RMSE': 3.4499810170312633, 'T_R2': 0.6570268782880198, 'T_

In [26]:
print("Writing report")
report.update({
    'tuner_duration': tuner_duration,
    'retrain_duration': retrain_duration,
    'total_duration': cut_duration + tuner_duration + retrain_duration,
    'error_results': error_results,
    'scaled_reduced_train_shape': scaled_reduced_train.shape,
    'best_trial_id': best_trial.trial_id,
    'best_trial_hyperparameters': best_trial.hyperparameters.values,
    'best_trial_score': best_trial.score,
    'best_forecaster_model': best_forecaster_model.summary(),
})

Writing report


In [27]:
report

{'execution_id': 'validate_pipeline_UCI_AIR_QUALITY_Window_L1_42',
 'timestamp': 'validate_pipeline',
 'cut_point_model': 'Window',
 'cut_point_method': 'L1',
 'cut_point_approach': 'Window L1',
 'seed': 42,
 'forecaster_model': 'LSTM',
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'AIR_QUALITY',
 'variables': ['C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH', 'AH'],
 'dataset_shape': (9357, 7),
 'train_shape': (7485, 7),
 'test_shape': (1872, 7),
 'cut_duration': 0.10567998886108398,
 'cut_point': '5385',
 'cut_point_perc': 71.9438877755511,
 'tuner_duration': 670.1102960109711,
 'retrain_duration': 60.539170026779175,
 'total_duration': 730.7551460266113,
 'error_results': {'Avg_MAPE': 735572565943.2211,
  'Avg_MAE': 32.998345925431174,
  'Avg_MSE': 6819.226338381778,
  'Avg_RMSE': 44.04242264408996,
  'Avg_R2': 0.40560638760242645,
  'Avg_WAPE': 0.3864814545225475,
  'C6H6(GT)_MAPE': 1.2469010335105057,
  'C6H6(GT)_MAE': 4.36893322

## 2. What would be the error if we predicted the average values for all variables (Dummy Forecaster)?

In [28]:
X_train, y_train = split_X_y(train)
X_test, y_test = split_X_y(test)

In [29]:
train_targets_flat = pd.DataFrame(y_train.reshape(-1, len(variables)), columns=variables)
avg_values = train_targets_flat.mean(axis=0).to_numpy()

In [30]:
n_test = y_test.shape[0]
dummy_pred = np.tile(avg_values, (n_test, FORECAST_HORIZON, 1))

dummy_pred_flat = dummy_pred.reshape(-1, len(variables))
y_test_flat = pd.DataFrame(y_test.reshape(-1, len(variables)), columns=variables)

In [31]:
dummy_error_results = get_error_results(y_test_flat, dummy_pred_flat, variables)
print(f"Error metrics for Dummy Forecaster (predicting average values): \n{dummy_error_results}")

Error metrics for Dummy Forecaster (predicting average values): 
{'Avg_MAPE': 8234635315137.34, 'Avg_MAE': 39.570501101478705, 'Avg_MSE': 9050.097810196155, 'Avg_RMSE': 54.308177300060635, 'Avg_R2': -1.304216974693167, 'Avg_WAPE': 0.4634554973283866, 'C6H6(GT)_MAPE': 2.4158267445655204, 'C6H6(GT)_MAE': 5.998942145850846, 'C6H6(GT)_MSE': 49.37824329877635, 'C6H6(GT)_RMSE': 7.026965440271949, 'C6H6(GT)_R2': -0.18020386569282976, 'C6H6(GT)_WAPE': 0.7497712562918357, 'NOx(GT)_MAPE': 0.69058204879659, 'NOx(GT)_MAE': 152.8286100552228, 'NOx(GT)_MSE': 49209.25736669915, 'NOx(GT)_RMSE': 221.8315968627985, 'NOx(GT)_R2': -0.12350328403007538, 'NOx(GT)_WAPE': 0.5073956992631623, 'NO2(GT)_MAPE': 0.38503788600654404, 'NO2(GT)_MAE': 53.84588657162112, 'NO2(GT)_MSE': 4627.602712434949, 'NO2(GT)_RMSE': 68.02648537470498, 'NO2(GT)_R2': -0.5820884482229336, 'NO2(GT)_WAPE': 0.3773222637098321, 'T_MAPE': 49407811890819.55, 'T_MAE': 10.715515800495881, 'T_MSE': 141.10016475956155, 'T_RMSE': 11.878559035487

In [32]:
df_comparison = pd.DataFrame({
    "Trained Model": pd.Series(error_results),
    "Dummy Forecaster": pd.Series(dummy_error_results)
})

df_comparison = df_comparison.round(5)
df_comparison

,Trained Model,Dummy Forecaster
Avg_MAPE,7.355726e+11,8.234635e+12
Avg_MAE,3.299835e+01,3.957050e+01
Avg_MSE,6.819226e+03,9.050098e+03
Avg_RMSE,4.404242e+01,5.430818e+01
Avg_R2,4.056100e-01,-1.304220e+00
Avg_WAPE,3.864800e-01,4.634600e-01
C6H6(GT)_MAPE,1.246900e+00,2.415830e+00
C6H6(GT)_MAE,4.368930e+00,5.998940e+00
C6H6(GT)_MSE,3.476876e+01,4.937824e+01
C6H6(GT)_RMSE,5.896500e+00,7.026970e+00
